In [16]:
""" Import Statments:
"""
import pandas as pd

In [17]:
pwd

'/Users/jorge/Med-Cabinet-2/Data'

In [18]:
ls

Med_Cab_1.4.ipynb  Med_Cab_1.6.ipynb  medcab.csv
Med_Cab_1.5.ipynb  cannabis.csv       medcab1_2.csv


In [39]:
df_X = pd.read_csv('/Users/jorge/Med-Cabinet-2/Data/medcab1_2.csv')
df_y = pd.read_csv('/Users/jorge/Med-Cabinet-2/Data/cannabis.csv')
df_X = df_X.fillna('none')
df_X = df_X.drop('Unnamed: 0', 1)

In [20]:
df_y['ID'] = df_y['Strain'].index
df_y['ID'] = df_y['ID'].astype(int)

In [21]:
df_X.shape, df_y.shape

((2350, 74), (2351, 7))

In [22]:
df_y.head(2)

,Strain,Type,Rating,Effects,Flavor,Description,ID
0,100-Og,hybrid,4.0,"Creative,Energetic,Tingly,Euphoric,Relaxed","Earthy,Sweet,Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,0
1,98-White-Widow,hybrid,4.7,"Relaxed,Aroused,Creative,Happy,Energetic","Flowery,Violet,Diesel",The ‘98 Aloha White Widow is an especially pot...,1


In [23]:
df_y = df_y.drop(columns =['Strain',
                           'Type',
                           'Rating',
                           'Effects',
                           'Flavor',
                           'Description'])

In [24]:
df_y.shape

(2351, 1)

In [25]:
arr_y = df_y.to_numpy()

In [26]:
arr_y = arr_y.astype(int)

In [27]:
arr_y.shape

(2351, 1)

In [28]:
type(arr_y)

numpy.ndarray

In [30]:
df_X['ID'].nunique()

2350

In [40]:
df_X.head(2)

,index,Strain,Type,Rating,Effects,Flavor,Description,Ammonia_flavor,Apple_flavor,Apricot_flavor,...,hybrid_type,sativa_type,indica_type,Savor,Style,Breed,Text,Effects_and_Flavor,ID,tokens
0,0,100-Og,hybrid,4.0,"Creative, Energetic, Tingly, Euphoric, Relaxed","Earthy, Sweet, Citrus",$100 OG is a 50/50 hybrid strain that packs a ...,False,False,False,...,True,False,False,", Earthy, Sweet, Citrus",", $100 OG is a 50/50 hybrid strain that packs ...",", 100-Og",Creative Energetic Tingly Euphoric Relaxed...,"Creative, Energetic, Tingly, Euphoric, Relaxed...",0,"['creative', 'energetic', 'tingly', 'euphoric'..."
1,1,98-White-Widow,hybrid,4.7,"Relaxed, Aroused, Creative, Happy, Energetic","Flowery, Violet, Diesel",The ‘98 Aloha White Widow is an especially pot...,False,False,False,...,True,False,False,", Flowery, Violet, Diesel",", The ‘98 Aloha White Widow is an especially p...",", 98-White-Widow",Relaxed Aroused Creative Happy Energetic ...,"Relaxed, Aroused, Creative, Happy, Energetic, ...",1,"['relaxed', 'aroused', 'creative', 'happy', 'e..."


In [32]:
df_X = df_X.drop(columns=['index', 
                          'Strain',
                          'Type',
                          'Rating', 
                          'Effects', 
                          'Flavor', 
                          'Description',
                          'ID'])    

In [36]:
df_X = df_X.replace([True, False],
                    [1.0, 0.0])

In [37]:
df_X.head(1)

,Unnamed: 0,Ammonia_flavor,Apple_flavor,Apricot_flavor,Berry_flavor,Blue_flavor,Blueberry_flavor,Citrus_flavor,Cheese_flavor,Chemical_flavor,...,None_effect,hybrid_type,sativa_type,indica_type,Savor,Style,Breed,Text,Effects_and_Flavor,tokens
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,...,0.0,1.0,0.0,0.0,", Earthy, Sweet, Citrus",", $100 OG is a 50/50 hybrid strain that packs ...",", 100-Og",Creative Energetic Tingly Euphoric Relaxed...,"Creative, Energetic, Tingly, Euphoric, Relaxed...","['creative', 'energetic', 'tingly', 'euphoric'..."


In [41]:
arr_X = df_X.to_numpy()

In [45]:
arr_X = arr_X.astype(int)

ValueError: invalid literal for int() with base 10: '100-Og'